# Selenium Driver 를 이용해서 다운로드 
SSS Instagram Web Service  
https://sssinstagram.com/  

# Setup Environment
References:  


# 패키지/모듈 로딩

In [1]:
import re 
from time import sleep

from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
driver = webdriver.Chrome()
driver 

<selenium.webdriver.chrome.webdriver.WebDriver (session="7f664f06ba67d74bb82be419434e223b")>

In [19]:
driver.get("https://sssinstagram.com/")

### 컨텐츠타입 입력

In [27]:
def select_contents_type(type='Video'):
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "pageTunerWrapper__items"))
        )
    finally:
        print('results:', results)
        a = results.find_element(By.LINK_TEXT, type.capitalize())
        print('a:', a)

        # app = driver.find_element(By.ID, 'app')
        # nav = app.find_element(By.CLASS_NAME, "pageTunerWrapper")
        # print('nav:', nav)

        # type = type.capitalize()
        # print('type:', type)
        # a = nav.find_element(By.LINK_TEXT, type)
        # print('a:', a)
        # a.click()
    

select_contents_type('video')

results: <selenium.webdriver.remote.webelement.WebElement (session="7f664f06ba67d74bb82be419434e223b", element="f.F2FBCFF3E0A1C08F4411C95718CEFB6B.d.D12F9004DF1493893C1BAA5DD7246C5D.e.13306")>


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"Video"}
  (Session info: chrome=127.0.6533.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7D9379642+30946]
	(No symbol) [0x00007FF7D932E3D9]
	(No symbol) [0x00007FF7D9226FDA]
	(No symbol) [0x00007FF7D927822C]
	(No symbol) [0x00007FF7D927850C]
	(No symbol) [0x00007FF7D926BCDC]
	(No symbol) [0x00007FF7D929CAAF]
	(No symbol) [0x00007FF7D926BBB6]
	(No symbol) [0x00007FF7D929CC80]
	(No symbol) [0x00007FF7D92BB041]
	(No symbol) [0x00007FF7D929C813]
	(No symbol) [0x00007FF7D926A6E5]
	(No symbol) [0x00007FF7D926B021]
	GetHandleVerifier [0x00007FF7D94AF84D+1301229]
	GetHandleVerifier [0x00007FF7D94BBDC7+1351783]
	GetHandleVerifier [0x00007FF7D94B2A13+1313971]
	GetHandleVerifier [0x00007FF7D93ADD16+245686]
	(No symbol) [0x00007FF7D933759F]
	(No 

### 링크주소 입력

In [7]:
def input_link(url):
    input_box = driver.find_element(By.ID, 'input')
    # print(input_box) 

    input_box.send_keys(url)
    input_box.send_keys(Keys.RETURN)

input_link("https://www.instagram.com/p/C-hts7migZz/?utm_source=ig_web_copy_link") 

### 다운로드

In [8]:
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=2)
import os 
from bs4 import BeautifulSoup
import requests 
from moviepy.editor import VideoFileClip, concatenate_videoclips

def get_download_urls():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    app = soup.find(id='app')
    result = app.find(class_="search-result")
    _list = result.find('ul', class_="output-list__list")
    # print(_list.prettify())
    urls = []
    for li in _list.find_all('li'):
        d = li.find('a').attrs
        urls.append(d['href'])
    return urls 


def download_file(url, local_filename):
    # Send a GET request to the URL
    with requests.get(url, stream=True) as response:
        response.raise_for_status()  # Check for HTTP request errors
        # Open a local file with write-binary mode
        with open(local_filename, 'wb') as file:
            # Write the content of the response in chunks
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
    print(f"File downloaded: {local_filename}")


def download(delay):
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "search-result"))
        )
    finally:
        print('results', results)

        # ul = results.find_element(By.CLASS_NAME, 'output-list__list')
        # print(ul)
        try:
            ul = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "output-list__list"))
            )
        finally:
            print('ul', ul)

            # 방법.1
            # for i, li in enumerate(ul.find_elements(By.TAG_NAME, 'li'), start=1):
            #     print([i, li])
            #     li.find_element(By.LINK_TEXT, 'Download').click()
            #     sleep(delay)

            # 방법.2
            urls = get_download_urls()
            filepaths = []
            for i, url in enumerate(urls, start=1):
                print([i, url])
                _dir = 'C:\\Users\\innovata\\Downloads'
                filepath = os.path.join(_dir, str(i).zfill(2)+'.mp4')
                filepaths.append(filepath)
                download_file(url, filepath)

            print('다운로드 완료')
            
            # 동영상 파일 병합
            clips = []
            for filepath in filepaths:
                clips.append(VideoFileClip(filepath))
            final_clip = concatenate_videoclips(clips)
            out_file = os.path.join(_dir, 'output.mp4')
            final_clip.write_videofile(out_file)

            # 다운로드한 분할 영상은 삭제 
            for filepath in filepaths:
                os.remove(filepath)

            print('DONE.')


# urls = get_download_urls()
# download(3)

In [105]:
from urllib.parse import urlparse
o = urlparse(urls[0])

In [109]:
o.path 

'/get'

In [11]:
def download_insta(url, type='video'):
    driver.get("https://sssinstagram.com/")
    select_contents_type(type)
    input_link(url) 
    download(delay=2)

download_insta("https://www.instagram.com/reel/C_Ln60OiEmy/?utm_source=ig_web_copy_link", type='video')

results <selenium.webdriver.remote.webelement.WebElement (session="7f664f06ba67d74bb82be419434e223b", element="f.F2FBCFF3E0A1C08F4411C95718CEFB6B.d.1FDD3CC8EF7D9324FB0A522B952F6371.e.8988")>


UnboundLocalError: local variable 'ul' referenced before assignment